Inference\
https://wikidocs.net/115055

https://huggingface.co/docs/transformers/index

http://kkma.snu.ac.kr/documents/index.jsp?doc=postag


# Author: Yoonhyuck WOO / JBNU_Industrial Information system Engineering
# Date; 2. 10. 2022 - 2. 22. 2022
# Title: Korean_NER
# Professor: Seung-Hoon Na

In [1]:
import os
import pandas as pd
import json
import transformers
from transformers import BertTokenizer
# from tqdm.auto import tqdm

In [2]:
PATH_dir = 'C:\\Users\\LG\\Desktop\\github\\JBNU-2022-SPRING\\English world class tagging & Korean_Named Entity Recognition\\Ko_En_NER_POStag_data\Ko_NER_POS'
file_name_ko_train = 'train.txt'
file_name_ko_test = 'test.txt'
file_name_ko_dev = 'dev.txt'

In [9]:
file_name = file_name_ko_test
PATH_data = os.path.join(PATH_dir, file_name)

# change file extention from '.txt' to '.json'
prepro_file_name = 'prepro_' + file_name.rsplit('.')[0] + '.json'
PATH_preprodata = os.path.join(PATH_dir, prepro_file_name)

In [10]:
with open(PATH_data, 'r', encoding='UTF8') as f:
    rawdata = f.readlines()

In [11]:
df0 = pd.DataFrame(rawdata)
df0

,0
0,박찬호/NNP\tNNP\tB\tB-PS\n
1,는/JX\tJX\tI\tO\n
2,8/SN\tSN\tB\tB-DT\n
3,일/NNB\tNNB\tI\tI-DT\n
4,(/SS\tSS\tI\tO\n
...,...
13768,것/NNB\tNNB\tB\tO\n
13769,이/VCP\tVCP\tI\tO\n
13770,다/EF\tEF\tI\tO\n
13771,./SF\tSF\tB\tO\n


In [14]:
df = pd.DataFrame(df0[0].str.split('/').tolist(),columns=['1-1','1-2','1-3'])
df

,1-1,1-2,1-3
0,박찬호,NNP\tNNP\tB\tB-PS\n,None
1,는,JX\tJX\tI\tO\n,None
2,8,SN\tSN\tB\tB-DT\n,None
3,일,NNB\tNNB\tI\tI-DT\n,None
4,(,SS\tSS\tI\tO\n,None
...,...,...,...
13768,것,NNB\tNNB\tB\tO\n,None
13769,이,VCP\tVCP\tI\tO\n,None
13770,다,EF\tEF\tI\tO\n,None
13771,.,SF\tSF\tB\tO\n,None


In [15]:
df2 = pd.DataFrame(df0[0].str.split('\t').tolist(),columns=['1','2','3','4'])
df2

,1,2,3,4
0,박찬호/NNP,NNP,B,B-PS\n
1,는/JX,JX,I,O\n
2,8/SN,SN,B,B-DT\n
3,일/NNB,NNB,I,I-DT\n
4,(/SS,SS,I,O\n
...,...,...,...,...
13768,것/NNB,NNB,B,O\n
13769,이/VCP,VCP,I,O\n
13770,다/EF,EF,I,O\n
13771,./SF,SF,B,O\n


In [16]:
df7 = pd.concat([df['1-1'],df2['4'].str.strip('\n')],axis = 1)
df7 = pd.DataFrame(df7)
df7 = df7.rename(columns={'1-1':'Entity','4':'tag'})
df7

,Entity,tag
0,박찬호,B-PS
1,는,O
2,8,B-DT
3,일,I-DT
4,(,O
...,...,...
13768,것,O
13769,이,O
13770,다,O
13771,.,O


In [17]:
print(len(df7))

13773


# Preprocessing 1

In [18]:
lst=[]
lst_set=[]
lst_ent=[]
lst_tag=[]
for i in range(6945):
    if df7['tag'][i] != None:
        lst_ent.append(df7['Entity'][i])
        lst_tag.append(df7['tag'][i])
    else:
        lst_set.append(lst_ent)
        lst_set.append(lst_tag)
        lst.append(lst_set)
        lst_ent = []
        lst_tag = []
        lst_set = [] 

In [22]:
print(lst[5][0])
print(lst[2][1])

['-', '주소창', '에', "'", '스포츠', "'", '만', '치', '시', '면', '스포츠서울닷컴', '기사', '가', '한눈', '에', '!']
['O', 'O', 'O', 'B-PS', 'I-PS', 'O', 'O', 'O']


In [25]:
PRETAINED_MODEL_NAME = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(PRETAINED_MODEL_NAME)

# Tokenizer tagging

In [26]:
lst=[]
lst_set = []
lst_ent = []
lst_tag = []
for i in range(len(df7)):
    if df7['tag'][i] != None:
        morph_to_tokens = tokenizer.tokenize(df7['Entity'][i])
        if len(df7['Entity'][i]) == len(morph_to_tokens):
            lst_ent.append(df7['Entity'][i])
            lst_tag.append(df7['tag'][i])
            
        else:
            if df7['tag'][i] == 'O':
                lst_ent.append(df7['Entity'][i])
                lst_tag.append(df7['tag'][i])
                for i in range(len(morph_to_tokens)-1):
                    lst_tag.append(df7['tag'][i])
                
            else:
                lst_ent.append(df7['Entity'][i])
                lst_tag.append(df7['tag'][i])
                split = df7['tag'][i].split('-')
                for i in range(len(morph_to_tokens)-1):
                    results = 'I-' + split[-1]
                    lst_tag.append(results)
            
    else:
        lst_ent = result = ' '.join(str(s) for s in lst_ent)
        lst_set.append(lst_ent)
        morph_to_tokens = tokenizer.tokenize(lst_ent)
        lst_set.append(morph_to_tokens)
        lst_set.append(lst_tag)
        lst.append(lst_set)
        result = 0
        lst_ent = []
        lst_tok = []
        lst_tag = []
        lst_set = [] 

In [27]:
print('entity', len(lst[3][0]))
print(lst[3][0])

print('token', len(lst[3][1]))
print(lst[3][1])

print('token_tag', len(lst[3][2]))
print(lst[3][2])

entity 15
- 5 번 마크 레이놀스 .
token 10
['-', '5', '번', '마', '##크', '레', '##이', '##놀', '##스', '.']
token_tag 6
['O', 'O', 'O', 'B-PS', 'I-PS', 'O']


In [30]:
print(lst[3])

['- 5 번 마크 레이놀스 .', ['-', '5', '번', '마', '##크', '레', '##이', '##놀', '##스', '.'], ['O', 'O', 'O', 'B-PS', 'I-PS', 'O']]


In [28]:
with open(PATH_preprodata, 'w') as f:
    json.dump(lst, f)

In [29]:
with open(PATH_preprodata, 'r') as f:
    preprodata = json.load(f)

# My Data

In [31]:
result = ' '.join(str(s) for s in lst[3][0])

In [32]:
inputs = tokenizer(result, return_tensors='pt')
print(inputs['input_ids'].size())
inputs

torch.Size([1, 12])


{'input_ids': tensor([[ 101,  118,  126, 9338, 9246, 9834, 9186, 9638, 9025, 9477,  119,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [33]:
from transformers import BertModel

In [34]:
PRETAINED_MODEL_NAME = 'bert-base-multilingual-cased'
bert = BertModel.from_pretrained(PRETAINED_MODEL_NAME)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [35]:
outputs = bert(**inputs)

In [36]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [37]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.3649, -0.0665, -0.3947,  ...,  0.2680,  0.1175,  0.1527],
         [ 0.1875, -0.6374,  0.4631,  ...,  0.2745, -0.1235,  0.9780],
         [-0.3853, -0.4464,  0.0116,  ..., -0.6496,  0.1723,  0.5837],
         ...,
         [ 0.9614, -0.3377, -0.0905,  ...,  0.4002,  0.0628,  0.3582],
         [ 0.7023, -0.3170, -0.7976,  ...,  0.3832,  0.1798,  0.4563],
         [ 0.7054, -0.0679, -0.3709,  ...,  0.1200,  0.0807,  0.4596]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[ 0.2764, -0.0306,  0.2660, -0.0536,  0.0713,  0.4242,  0.0637,  0.2402,
         -0.4094,  0.2865,  0.0693, -0.1161, -0.2005, -0.1117,  0.1129, -0.3604,
          0.7204,  0.0443,  0.0575, -0.3142, -1.0000, -0.1155, -0.3364, -0.1114,
         -0.4538, -0.0540, -0.0862,  0.1193,  0.1355, -0.0590, -0.0113, -1.0000,
          0.4654,  0.6495,  0.1981, -0.0345,  0.1674,  0.3721,  0.2504, -0.4007,
         -0.1429, -0.1354, -0.050